# Battery dispatch optimization
The goal of this exercise is to optimize the charge/discharge behavior of a battery system performing energy arbitrage in the NYISO market. The model objective is to maximize revenue over a year and the project objective is to gain insights into market dynamics and expected system behavior. We have hourly LBMP data for the entire year (taken from 2017), which we assume is accurate with perfect foresight.

The battery system has maximum storage capacity of 200 kWh and a power rating of 100 kW (charge and discharge). Round-trip AC-AC efficiency is 85%. The maximum daily discharge throughput is constrained to 200 kWh within a 24-hour period.

Because we are using this model to understand the market dynamics in the NYISO NYC hub, it's fine to run the entire year optimzation at once rather than running in discrete periods. When actually operating the battery we would need to use price forecasts that would become less accurate over time. In that situation a series of multi-day optimizations would be used. The first window would assume a starting charge state (I'm using half of the manimum charge in this exercise), and might predict 3-4 days. The first 24-hours would be kept and used as a constraint in the next period. For the second period, a window of 4-5 days would be used -- values from the first day would be used to constrain hours 0-23, and the optimization would again use a look-ahead of 2-3 days to ensure accurate model behavior. This cycle (keep 24 hours and use then as a constrain in the next iteration) would then continue through the end of the year.

I've elected to stick with the simplier approach here because it is easier/faster to code and should provide the same optimized dispatch results. Actually optimizing dispatch for a battery system would require forecasting prices. Because this is a much simpler exercise a simpler model that takes less time to build is fine.

In [39]:
import pandas as pd
from pathlib import Path
import altair as alt

# Select your appropriate notebook type for rendering Altair figures
alt.renderers.enable('jupyterlab')
# alt.renderers.enable('notebook')
alt.data_transformers.enable('default', max_rows=None)

from src.read_data import read_all_nyc
from src.read_all_NL import read_all_NL
from src.battery_model import optimize_year, model_to_df

## Read data
Functions to read data and run the optimization model are provided in scripts in the `src` folder. The `read_all_nyc` function combines data from daily .csv files, filters out all non-NYC node prices, and renames the columns to snake case.

In [40]:
# data_path = Path.cwd() / '2017_NYISO_LBMPs'
# df = read_all_nyc(data_path=data_path)

#data_path = Path.cwd() / 'Entsoe_prices_2023'
#df = read_entsoe_2023(data_path=data_path)

df = read_all_NL()

In [41]:
df.head()

,time_stamp,lbmp,hour
0,2023-01-01 00:00:00+01:00,-3.61,0
1,2023-01-01 01:00:00+01:00,-1.46,1
2,2023-01-01 02:00:00+01:00,-1.52,2
3,2023-01-01 03:00:00+01:00,-5.00,3
4,2023-01-01 04:00:00+01:00,-4.60,4


## Model parameters and constraints

**Parameters**
- $t$: timestep or hour
- $R_{max}$ (100 kW): maximum power than can be delivered to or from the battery (charge or discharge rate)
- $S_{max}$ (200 kWh): maximum battery capacity
- $S_t$: storage at time $t$
- Eff ($\eta$) (85%): efficiency 
- $D_{max}$ (200 kWh): max discharge within a 24 hour period
- $P_t$: LBMP at time $t$

**Decision variables**
- $E^{in}_t$: energy delivered to the battery at time $t$
- $E^{out}_t$: energy discharged from the battery at time $t$

**Constraints**
- $S_1$ = $\frac{S_{max}}{2}$ (Assume storage begins at half of capacity)
- $S_t$ = $S_{t-1} + \sqrt{\eta} \times E^{in}_{t-1} - \frac{E^{out}_{t-1}}{\sqrt{\eta}}$
- $\forall t, S_t \geq 0$
- $\forall t, S_t \leq S_{max}$
- $\forall t, E^{in}_t \leq R_{max}$
- $\forall t, E^{out}_t \leq R_{max}$
- $\forall t, E^{out}_t \leq S_t$
- $\sum_{t'=t-23}^t E^{out}_{t'} \leq D_{max} \forall t \subset (T, t \geq 24)$

## Run the optimization model
The `optimize_year` function takes in the LBMP data from our new dataframe and returns the optimization results in a dataframe.

In [42]:
results_df = optimize_year(df)

In [43]:
import pytz

# Ensure 'time_stamp' is in datetime format, and if it's timezone-aware, convert it to UTC first
results_df['time_stamp'] = pd.to_datetime(results_df['time_stamp'], utc=True)

# Convert to Amsterdam timezone
amsterdam_tz = pytz.timezone('Europe/Amsterdam')
results_df['time_stamp'] = results_df['time_stamp'].dt.tz_convert(amsterdam_tz)


results_df.head()

,hour,Ein,Eout,lbmp,charge_state,time_stamp
0,0,50.000000,0.0,-3.61,200.000000,2023-01-01 00:00:00+01:00
1,1,34.577517,50.0,-1.46,246.097722,2023-01-01 01:00:00+01:00
2,2,50.000000,50.0,-1.52,223.744003,2023-01-01 02:00:00+01:00
3,3,50.000000,0.0,-5.00,215.609111,2023-01-01 03:00:00+01:00
4,4,50.000000,0.0,-4.60,261.706833,2023-01-01 04:00:00+01:00


### Output results

In [44]:


results_df.to_csv('full_year_optimization_results.csv')

## Analysis of results
In this exercise I've been asked to present the following:
- Summary values
    - Annual revenue
    - Annual charging costs
    - Annual discharged throughput
- Plots
    - Hourly dispatch and LBMP for the most profitable week (assuming calendar week)
    - Total profit for each month

### Summary values
Revenue, costs, and profit still need to be calculated using energy in/out and the hourly price

In [45]:
# Convert $/MWh to $/kWh
results_df['revenue'] = results_df.Eout * results_df.lbmp / 1000
results_df['charge_cost'] = results_df.Ein * results_df.lbmp  / 1000
results_df['profit'] = results_df.revenue - results_df.charge_cost


In [46]:
total_revenue = results_df.revenue.sum()
total_charge_cost = results_df.charge_cost.sum()
total_discharge = results_df.Eout.sum()

total_charge_losses = results_df.Ein.sum() - results_df.Eout.sum()

print('Annual revenue was ${:,.0f}'.format(total_revenue))
print('Annual charging cost was ${:,.0f}'.format(total_charge_cost))
print('Annual discharged throughput was {:,.0f} kWh'.format(total_discharge))

avg_charge_price = total_charge_cost / total_discharge
avg_discharge_price = total_revenue / total_discharge

print('Annual average charged price was {:,.0f} $/MWh'.format(avg_charge_price*1000))
print('Annual average discharged price was {:,.0f} $/MWh'.format(avg_discharge_price*1000))

print('Cycles per year: {:,.0f}'.format(total_discharge/400)) #TODO: make battery size 200kWh parameterized

Annual revenue was $17,382
Annual charging cost was $9,928
Annual discharged throughput was 135,646 kWh
Annual average charged price was 73 $/MWh
Annual average discharged price was 128 $/MWh
Cycles per year: 339


### Figures

In [47]:
results_df['week'] = results_df.time_stamp.dt.isocalendar().week
results_df['month'] = results_df.time_stamp.dt.month
results_df['hour_of_day'] = results_df.time_stamp.dt.hour


After dropping the first day of 2023 just like 2017 (which was a Sunday, and part of the 52nd week of 2016), the most profitable week was the last week of the year.

In [48]:
results_df.loc[results_df.time_stamp >= '2023-01-02', :].groupby('month')['profit'].sum().idxmax()

7

Including both the dispatch and hourly Day-Ahead EPEX prices (in NY:LBMP) in a single plot is difficult because their values have different scales (dispatch is capped at 100 kWh while LBMP in week 52 goes over \\$200/kWh). A dual y-axis plot can be difficult to read, so I've decided to show one plot on top of the other. The plot of LBMP uses color to encode dispatch, which helps to make the whole thing easier to interprete.

In [60]:
data = results_df.loc[(results_df.time_stamp >= '2023-02-01') &
                      (results_df.time_stamp <= '2023-02-03'), :].copy()

data.loc[:, 'dispatch'] = data.Ein - data.Eout

dispatch_data = pd.melt(data, id_vars='time_stamp', value_vars=['Eout', 'Ein'], var_name='Dispatch')

color_scale = alt.Scale(
            domain=['Ein', 'Eout'],
            range=['#f99820', '#2081f9']
        )

dispatch = alt.Chart(dispatch_data).mark_line().encode(
    x='time_stamp:T',
    y=alt.Y('value:Q', axis=alt.Axis(title='Electricity in/out (kWh)')),
    color=alt.Color('Dispatch:N', scale=color_scale)
).properties(
    height=200
)

lbmp = alt.Chart(data).mark_circle().encode(
    x='time_stamp:T',
    y=alt.Y('lbmp:Q', axis=alt.Axis(title='NL Day-Ahead €/MWh')),
    color=alt.Color('dispatch:Q', scale=alt.Scale(scheme='blueorange')),
    tooltip='dispatch:Q'
).properties(
    height=200
)

alt.vconcat(
    dispatch,
    lbmp
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [50]:
alt.Chart(results_df).mark_line().encode(
    x='month:O',
    y='sum(profit):Q'
).properties(
    height=300,
    width=500
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
